In [ ]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
spark_version = 'spark-3.0.3'
# spark_version = 'spark-3.<enter version>'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic 

In [ ]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2021-12-21 07:52:06--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  --.-KB/s    in 0.08s   

2021-12-21 07:52:06 (10.3 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("bigdata").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [ ]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Books_v1_01.tsv.gz"
spark.sparkContext.addFile(url)

df = spark.read.csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   22480053|R28HBXXO1UEVJT|0843952016|      34858117|          The Rising|           Books|          5|            0|          0|   N|                N|Great Twist on Zo...|I've known about ...| 2012-05-03|
|         US|   44244451| RZKRFS2UUMFFU|031088926X|     676347131|Sticky Faith Teen...|           Books|          5|    

In [ ]:
df.count()

6448650

In [32]:
#Pull Data Review ID table 

from pyspark.sql.functions import to_date

review_id = df.select(["review_id", "customer_id", "product_id", "product_parent", to_date("review_date", 'yyy-MM-dd').alias("review_date")]).dropDuplicates(["review_date"])
review_id.show()

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
| R4CVFBLWPYH3V|   49873420|B00002S5IG|     496442757| 2000-03-17|
|R1WJLYHB20FB8X|   49240049|B00004R8KU|     363727888| 2000-07-03|
|R3EGDUOXBFBJGM|   46117150|B00004TYCR|     625957090| 2000-12-26|
| RNIK3ETKB3CDP|   47306102|B00004W62O|     832691196| 2001-05-16|
|R2GW4EEUQUNWHY|   39916176|B00005S8KN|     778442729| 2002-03-02|
|R3821XP3LM7J53|   50409913|B00005NZDX|     403249579| 2002-06-20|
| RH6LJNZOJN11S|   34546473|B00005MNH9|     848605208| 2002-12-06|
|R3EJ747INY7P76|   49011583|B00006JJNE|     656523758| 2002-12-25|
|R2MDF2WTRJ94U7|   52288732|B00068IBZW|     246047200| 2005-01-16|
| RA6X65LZ7RRNF|   52982080|B0000DC67P|     902063256| 2005-06-06|
| RHPAY6OD2MAO8|   10677949|1424115434|      40686026| 2006-05-17|
|R14JZ1C1GA8JWL|   49655548|1573225541|     223433193| 2006-05

In [ ]:
#Data Review Count 
review_id.count()

5776

In [ ]:
#Pull products Table 
products_df = df.select(["product_id","product_title"]).dropDuplicates(subset = ["product_id"])
products_df.show()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|0001714422|'C' Is for Clown ...|
|0002216973|     Red Adam's Lady|
|000255111X|The Adventures of...|
|0004708075|The Complete Nove...|
|0004708237|Collins Gem Shubu...|
|0006162754|After the Funeral...|
|0006393020|Firewing (Silverw...|
|0006476155| Along Came a Spider|
|0006479529|   DRUID'S TUNE, The|
|0006544150|           In Xanadu|
|0006550479|   The Hottest State|
|0006943543|Precalculus: Math...|
|0007102712|Tiny Trumpet Play...|
|0007121040|Miss Marple's Fin...|
|0007162928|The Lion, the Wit...|
|0007163932|The Highly Sensit...|
|0007166710|The Ultimate " Si...|
|0007175698|The Mirror Crack'...|
|000717764X|Duck's Key: Where...|
|0007178484|Reading "Lolita" ...|
+----------+--------------------+
only showing top 20 rows



In [ ]:
# Products Table count 
products_df.count()

1531025

In [ ]:
#Customer table 
customer_df = df.groupby("customer_id").agg({"customer_id": "count"}).withColumnRenamed("count(customer_id)", "customer_count").dropDuplicates(subset = ["customer_id"])
customer_df.show()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   47098686|             1|
|    9973109|             1|
|   52477231|            28|
|   27568383|             1|
|   15519001|             2|
|   52130093|             2|
|   48794123|           102|
|   12453285|             1|
|   22824019|             1|
|   21087503|             9|
|   52553295|             4|
|   44143172|             1|
|   47844321|             3|
|   40575930|             1|
|   12657201|             1|
|   38674185|             1|
|   40325486|           166|
|   42645542|             2|
|   18768108|             5|
|   10546543|           361|
+-----------+--------------+
only showing top 20 rows



In [ ]:
#Customer table Count 
customer_df.count()

2946996

In [ ]:
#Pull Vine Table 
vine_table = df.select(["review_id","star_rating","helpful_votes","total_votes","vine"]).dropDuplicates(subset = ["review_id"])
vine_table.show()

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R10076S8MIEFT1|          2|           11|         23|   N|
|R1007VPNHEK8DU|          5|           11|         14|   N|
|R100QWXHWZLV4R|          4|            0|          0|   N|
|R100RHJPDHTSY7|          3|            1|          1|   N|
|R100YM0MDZEZ6A|          5|            2|          3|   N|
|R100ZY5E77EQFG|          5|            2|          3|   N|
|R1015GKMBBH7WM|          5|            6|          6|   N|
|R1015KRO2UAZ3O|          5|            2|          2|   N|
|R1018UVQG92B1Z|          2|            7|         10|   N|
|R101B0UWLYMKC6|          5|           28|         37|   N|
|R101G8Y47QYC64|          5|            1|          1|   N|
|R101P2PC2MUDYB|          4|            0|          0|   N|
|R101V5OJ06Q8JT|          5|            0|          0|   N|
|R101X2BTWNLZWT|          5|            

In [ ]:
#count Vine Table 
vine_table.count()

6448650

In [ ]:
# Write to RDS
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://<endpoint>:5432/bigdata01"
config = {"user":"postgres", "password": "", "driver":"org.postgresql.Driver"}

In [ ]:
# Write review_id to table in RDS
review_id.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [ ]:
 # Write products_df to table in RDS
products_df.write.jdbc(url=jdbc_url, table='products', mode=mode, properties=config)

In [ ]:
# Write customer_df to table in RDS
customer_df.write.jdbc(url=jdbc_url, table='customers', mode=mode, properties=config)

In [ ]:
# Write vine_df to table in RDS
vine_table.write.jdbc(url=jdbc_url, table='vines', mode=mode, properties=config)